In [ ]:
!pip install plotly
import os
import sys
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.integrate import odeint
import plotly.graph_objects as go
# import plotly.io as pio
# pio.renderers.default = "notebook"
# %matplotlib inline
# plt.style.use('ggplot')
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import colors


In [ ]:
# Jupyter Specifics
!pip install ipywidgets
from IPython.display import HTML
from ipywidgets.widgets import interact, IntSlider, FloatSlider, Layout

style = {'description_width': '100px'}
slider_layout = Layout(width='99%')

In [ ]:
start_day = 25
end_day = 39
lockdown = True

In [ ]:
def ode_model(z, t, beta, sigma, gamma):
    """
    ode_model calculate the values for equation.

    :param z: array of [initS, initE, initI, initR]
    :param t: time
    :param beta: Infection rate
    :param sigma: Incubation rate
    :param gamma: Recovery rate
    :return: array of derivative values [dSdt, dEdt, dIdt, dRdt]
    """ 
    S, E, I, R = z
    N = S + E + I + R
    
    if lockdown:
        dSdt = -get_beta(t)*S*I/N
        dEdt = get_beta(t)*S*I/N - sigma*E
    else:
        dSdt = -beta*S*I/N
        dEdt = beta*S*I/N - sigma*E
    dIdt = sigma*E - gamma*I
    dRdt = gamma*I
    return [dSdt, dEdt, dIdt, dRdt]

In [ ]:
def R_0(t):
    """
    R_0 function to calculate varying value of R

    :param t: time (day)
    :return: new value of R_0
    """ 
    if t > start_day and t < end_day:
        return 2.2
    else:
        return 3.8

def get_beta(t):
    """
    get_beta function to calculate new value of beta based on condition

    :param t: time (day)
    :return: new value of beta
    """
    return R_0(t) * gamma

In [ ]:
def ode_solver(t, initial_conditions, params):
    initE, initI, initR, initN = initial_conditions
    beta, sigma, gamma = params
    initS = initN - (initE + initI + initR)
    res = odeint(ode_model, [initS, initE, initI, initR], t, args=(beta, sigma, gamma))
    return res

In [ ]:
initN = 10000
initE = 1
initI = 5
initR = 0
sigma = 1/5.2
gamma = 1/2.9
R0 = 3.8
beta = R0 * gamma
days = 100

In [ ]:
plt.rcParams['savefig.facecolor']='white'

def main(initE, initI, initR, initN, beta, sigma, gamma, days):
    initial_conditions = [initE, initI, initR, initN]
    params = [beta, sigma, gamma]
    tspan = np.arange(0, days, 1)
    sol = ode_solver(tspan, initial_conditions, params)
    S, E, I, R = sol[:, 0], sol[:, 1], sol[:, 2], sol[:, 3]
    

    simulation = pd.DataFrame({"susceptible":S, "exposed":E, 
                "infectious":I, "removed":R})
    color_list = ["blue","red","green","purple"]

    sns.lineplot(data=simulation,palette=color_list).set(title = 'SEIR Model', xlabel = 'Days', ylabel = 'Population')

    if lockdown:
        plt.axvline(start_day, linestyle="--", color='gray')
        plt.axvline(end_day, linestyle="--", color='gray')

    if not os.path.exists("images"):
        os.mkdir("images")
    plt.savefig("images/seir_simulation.png", bbox_inches="tight")


In [ ]:
interact(main, initE=IntSlider(min=0, max=100000, step=1, value=initE, description='initE', style=style, layout=slider_layout),
               initI=IntSlider(min=0, max=100000, step=10, value=initI, description='initI', style=style, layout=slider_layout),
               initR=IntSlider(min=0, max=100000, step=10, value=initR, description='initR', style=style, layout=slider_layout),
               initN=IntSlider(min=0, max=1380000000, step=1000, value=initN, description='initN', style=style, layout=slider_layout),
               beta=FloatSlider(min=0, max=4, step=0.01, value=beta, description='Infection rate', style=style, layout=slider_layout),
               sigma=FloatSlider(min=0, max=4, step=0.01, value=sigma, description='Incubation rate', style=style, layout=slider_layout),
               gamma=FloatSlider(min=0, max=4, step=0.01, value=gamma, description='Recovery rate', style=style, layout=slider_layout),
               days=IntSlider(min=1, max=600, step=7, value=days, description='Days', style=style, layout=slider_layout)
        )